In [ ]:
%matplotlib inline
%load_ext line_profiler
import numpy as np
import pandas as pd
import math
import scipy
import scipy.stats as ss
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.animation as animation
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rc
from matplotlib.patches import Rectangle
import seaborn.apionly as sns
from IPython.display import HTML
import os, glob, sys
import warnings
warnings.simplefilter('always')

from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.plotting import figure 
output_notebook()

In [ ]:
np.seterr(all='raise', divide='ignore', over='raise', under='ignore', invalid='raise')

#### Load Generated Synthetic Data

In [ ]:
data = pd.read_csv('observation_single_particle.csv')
data = data.drop(['Unnamed: 0'],axis=1)

In [ ]:
%matplotlib inline
plt.plot(data['0'],data['1'])

N is Number of time levels and is obtained from the length of observations

In [ ]:
Wn = np.array(data['1'])
N = len(Wn)  

In [ ]:
# ffbs = FFBS()
# A = np.zeros([4,N], dtype=np.float64)
# # Likelihood or Emission distribution
# ID = 0
# p1 = hyper_param['gama']/(hyper_param['gama']+M-1)
# pi0 = np.array([p1,0,0,0,1-p1,0,0,0]) 
# Snm = np.column_stack([[1,1,1,1,0,0,0,0],[0,1,2,3,0,1,2,3]]) #Snm[8,2]
# F = ffbs.emission_distribution(Wn,Sn,params,M,ID,Snm)      # F[8,N]
# pi_filter = update_pi_filter(pi)

In [ ]:
class FFBS:
    def emission_distribution(self,x,Sn,params,M,ID,Snm):
        # This is needed for the calculation of the Filter
        ind = list(range(M))
        ind.remove(ID)
        self.Ksn = np.where(Sn==1,params['k_sigma2'],0.0) # Ksn[M,N]
        self.Ksn_filter = [0.0,params['k_sigma2'],0.0,0.0,0.0,params['k_sigma2'],0.0,0.0] # Ksn_filter[8]
        #self.rate [8,N]
        r = params['theta']*hyper_param['dt']*(params['rho']+np.dot(params['bm'][0,ind],self.Ksn[ind,:]))
        self.rate = [r+params['theta']*hyper_param['dt']*self.Ksn_filter[i] for i in range(2)]   
        likelihood = [-self.rate[0]+x*np.log(self.rate[0]),-self.rate[1]+x*np.log(self.rate[1])]
        
        d = []
        d.append(likelihood)
        for i in range(6):
            d.append(likelihood[0])         
        
        return np.vstack(d)
    
    def get_filter(self,A,pi0,pi_filter,N,F):
        A[:,0] = np.log(pi0)
        #=============================================================================================
        for i in range(1,2):
            A[0,i] = F[0,i]+A[0,i-1]+np.log(pi_filter[0,0])
            #-------------------------------------------------------------------------------------
            MM = np.array([(A[0,i-1]+np.log(pi_filter[0,1])),(A[1,i-1]+np.log(pi_filter[1,1])),(A[2,i-1]+np.log(pi_filter[2,1]))])
            Max = -np.max(MM)
            A[1,i] = F[1,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[1,i-1]+np.log(pi_filter[1,2])),(A[2,i-1]+np.log(pi_filter[2,2]))])
            A[2,i] = F[2,i]+np.log(np.sum(np.exp(MM)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[1,i-1]+np.log(pi_filter[1,3])),(A[3,i-1]+np.log(pi_filter[3,3]))])
            A[3,i] = F[3,i]+np.log(np.sum(np.exp(MM)))
            #--------------------------------------------------------------------------------------                
            A[4,i] = F[4,i]+A[4,i-1]+np.log(pi_filter[4,4])
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[4,i-1]+np.log(pi_filter[4,5])),(A[5,i-1]+np.log(pi_filter[5,5])),(A[6,i-1]+np.log(pi_filter[6,5]))])
            A[5,i] = F[5,i]+np.log(np.sum(np.exp(MM)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[5,i-1]+np.log(pi_filter[5,6])),(A[6,i-1]+np.log(pi_filter[6,6]))])
            A[6,i] = F[6,i]+np.log(np.sum(np.exp(MM)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[5,i-1]+np.log(pi_filter[5,7])),(A[7,i-1]+np.log(pi_filter[7,7]))])
            A[7,i] = F[7,i]+np.log(np.sum(np.exp(MM)))
        #=============================================================================================
        for i in range(2,N):
            A[0,i] = F[0,i]+A[0,i-1]+np.log(pi_filter[0,0])
            #-------------------------------------------------------------------------------------
            MM = np.array([(A[0,i-1]+np.log(pi_filter[0,1])),(A[1,i-1]+np.log(pi_filter[1,1])),(A[2,i-1]+np.log(pi_filter[2,1]))])
            Max = -np.max(MM)
            A[1,i] = F[1,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[1,i-1]+np.log(pi_filter[1,2])),(A[2,i-1]+np.log(pi_filter[2,2]))])
            Max = -np.max(MM)
            A[2,i] = F[2,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[1,i-1]+np.log(pi_filter[1,3])),(A[3,i-1]+np.log(pi_filter[3,3]))])
            Max = -np.max(MM)
            A[3,i] = F[3,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------                
            A[4,i] = F[4,i]+A[4,i-1]+np.log(pi_filter[4,4])
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[4,i-1]+np.log(pi_filter[4,5])),(A[5,i-1]+np.log(pi_filter[5,5])),(A[6,i-1]+np.log(pi_filter[6,5]))])
            Max = -np.max(MM)
            A[5,i] = F[5,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[5,i-1]+np.log(pi_filter[5,6])),(A[6,i-1]+np.log(pi_filter[6,6]))])
            Max = -np.max(MM)
            A[6,i] = F[6,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
            #--------------------------------------------------------------------------------------
            MM = np.array([(A[5,i-1]+np.log(pi_filter[5,7])),(A[7,i-1]+np.log(pi_filter[7,7]))])
            Max = -np.max(MM)
            A[7,i] = F[7,i]-Max+np.log1p(np.sum(np.exp(np.delete(MM,np.argmax(MM))+Max)))
        
        return A
    
    # Filter changes for each particle because of the change in the rates. But transition matrix is the same.
    def Filter(self,pi,pi_filter,N,ID,Sn,Snm,Wn,M,params,hyper_param):  
        A = np.zeros([8,N], dtype=np.float64)
        # Likelihood or Emission distribution
        F = self.emission_distribution(Wn,Sn,params,M,ID,Snm)      # F[8,N]
        
        p1 = hyper_param['gama']/(hyper_param['gama']+M-1)
        pi0 = np.array([p1,0,0,0,1-p1,0,0,0])   # pi0[8,1] 
#         A_star = pi0*F[:,0]
#         c1 = np.sum(A_star)
#         A[:,0] = A_star[:]/c1  # Array of 8*N where 8 is the number of possible states for the filter 
    
        return self.get_filter(A,pi0,pi_filter,N,F)
    
    def _FFBS(self,N,pi,pi_filter,params):
        # States of the system considered for the filter
        Snm = np.column_stack([[1,1,1,1,0,0,0,0],[0,1,2,3,0,1,2,3]]) #Snm[8,2]
        bm = np.zeros([1,M],dtype=int) 
        for ID in range(M):      
            A = self.Filter(pi,pi_filter,N,ID,Sn,Snm,Wn,M,params,hyper_param) # A[8,N]
            # Sanity Check
            if np.sum(A,axis=0).all()==1:
                pass
            else:
                raise Exception('Filter is not working for ID{} with {}'.format(ID,np.sum(A,axis=0)))        
            
            # Sampling via Gumbel distribution
            g_i = -np.log(-np.log(np.random.uniform(size=8)))
            ind_N = np.argmax(g_i+A[:,N-1])          # Get the state at the last time level
            # Backward Sampling
            Sn[ID,N-1] = Snm[ind_N,1]
            bm[0,ID] = Snm[ind_N,0]
            AA = A[:4,:] if ind_N<4 else A[4:,:]     # AA[4,N]           
            for n in range(N-2,-1,-1):
                weight = AA[:,n]+np.log(pi[:,Sn[ID,n+1]])
                g_i = -np.log(-np.log(np.random.uniform(size=4)))
                Sn[ID,n] = np.argmax(g_i+weight) 
          
        return bm, Sn

In [ ]:
class Metropolis_Hastings:
    def __init__(self,alfa1,alfa2):
        self.alfa1 = alfa1
        self.alfa2 = alfa2
        
    def _proposal(self,x,alfa):
        return np.random.gamma(alfa,scale=x/alfa) 
    
    def sample(self,rho_old,k_sigma2_old):
        rho_prop = self._proposal(rho_old,self.alfa1) 
        k_sigma2_prop = self._proposal(k_sigma2_old,self.alfa2)
        return rho_prop, k_sigma2_prop
    
    def _proposal_pdf(self,x1,x2,alfa):
        return ((2*alfa-1)*(np.log(x1)-np.log(x2))+alfa*(x2/x1-x1/x2))
    
    def _prior(self,x1,x2,alfa,beta):
        return (alfa-1)*(np.log(x2)-np.log(x1))+(x1-x2)/beta
    
    def _likelihood(self,x1,y1,x2,y2,Sn,params,hyper_param,Wn):
        Ksn1 = np.where(Sn==1,y1,0.0)
        rate1 = params['theta']*(x1+np.dot(params['bm'],Ksn1))*hyper_param['dt'] 
        
        Ksn2 = np.where(Sn==1,y2,0.0)
        rate2 = params['theta']*(x2+np.dot(params['bm'],Ksn2))*hyper_param['dt'] 

        return np.sum(Wn*(np.log(rate1)-np.log(rate2))+(rate2-rate1))
    
    def Acceptance_ratio(self,params,hyper_param,rho_prop,k_sigma2_prop,rho_old,k_sigma2_old,Wn):        
        # Calculated the log of Prior_ratio, Proposal_ratio, and Likelihood_ratio
        prior_ratio = (self._prior(rho_old,rho_prop,hyper_param['alfa1_prior'],hyper_param['beta1'])+
                       self._prior(k_sigma2_old,k_sigma2_prop,hyper_param['alfa2_prior'],hyper_param['beta2']))         
        
        proposal_ratio = (self._proposal_pdf(rho_old,rho_prop,self.alfa1)+
                          self._proposal_pdf(k_sigma2_old,k_sigma2_prop,self.alfa2))
        
        likelihood_ratio = self._likelihood(rho_prop,k_sigma2_prop,rho_old,k_sigma2_old,Sn,params,hyper_param,Wn) 
        
        return likelihood_ratio+prior_ratio+proposal_ratio

In [ ]:
class Gibbs_sampler:  
    def update_theta(self,alfa,beta,Wn,Ksn,params):
        alfa_new = alfa+np.sum(Wn)
        beta_new = beta+np.sum((params['rho']+np.dot(params['bm'][0,:],Ksn))*hyper_param['dt'])
        
        return np.random.gamma(alfa_new,1./beta_new)
    
    def update_roh_ksigma2(self,params,repeats,accepted):
        MH = Metropolis_Hastings(hyper_param['alfa1_proposal'],hyper_param['alfa2_proposal'])
        
        for i in range(25):
            rho_old = params['rho']
            k_sigma2_old = params['k_sigma2']

            # Step (a)
            rho_prop,k_sigma2_prop = MH.sample(rho_old,k_sigma2_old)
            # Step (b)
            AR = MH.Acceptance_ratio(params,hyper_param,rho_prop,k_sigma2_prop,rho_old,k_sigma2_old,Wn)
            # Step (c)
            v = np.random.exponential(scale=1)
            # Step (d)
            if v>-AR:
                accepted = accepted + 1.0
                params['rho'] = rho_prop
                params['k_sigma2'] = k_sigma2_prop
            else:
                params['rho'] = rho_old
                params['k_sigma2'] = k_sigma2_old
            
        return params['rho'], params['k_sigma2'], accepted
            
    def update_pi(self,Sn,M):
        # total number of transitions for all particles over time 
        # is M*(N-1)
        count = dict({0:{0:0,1:0,2:0,3:0},1:{0:0,1:0,2:0,3:0},
                      2:{0:0,1:0,2:0,3:0},3:{0:0,1:0,2:0,3:0}})
        for j in range(M):
            for i in range(N-1):
                state1 = Sn[j,i]
                if Sn[j,i+1]==0:
                    count[state1][0] += 1
                elif Sn[j,i+1]==1:
                    count[state1][1] += 1
                elif Sn[j,i+1]==2:
                    count[state1][2] += 1
                elif Sn[j,i+1]==3:
                    count[state1][3] += 1
                    
        if count[0][1]>M:
            raise Exception('Wrong transitions are happening')
            
        return count

    def update_weight_State(self,hyper_param,pi,pi_filter,params):
        ffbs = FFBS()
        return ffbs._FFBS(N,pi,pi_filter,params)

In [ ]:
def update_pi_filter(pi):
    aa = np.zeros([4,4],dtype=np.float64)
    d1 = np.column_stack([pi,aa])
    d2 = np.column_stack([aa,pi])
    return np.vstack([d1,d2])

In [ ]:
def __main__(repeats,M,pi,Sn,Ksn,handle):
    data1 = 'Wn_single3.h5'
    data2 = 'params_single3.h5'
    data3 = 'trajectory_single3.h5'
    
    data = dict({'theta': [],'rho': [],'k_sigma2': [],'B':[], 'ite':[],'E_b':[],'E_ph':[]})

    gibbs = Gibbs_sampler()
    accepted  = 0.0
    p = pd.DataFrame()
    for j in range(repeats):
        print(j)
        start1 =  time.time()
        params['theta'] = gibbs.update_theta(hyper_param['alfa'],hyper_param['beta'],Wn,Ksn,params)
        # Metropolis Hastings
        params['rho'], params['k_sigma2'], accepted = gibbs.update_roh_ksigma2(params,repeats,accepted)
        # Update Transition Probabilities
        count = gibbs.update_pi(Sn,M)

        state = 0 
        pi[0] = np.array(list(np.random.dirichlet((0.7+count[state][0],0.3+count[state][1])))+[0,0])
        state = 1
        pi[1] = np.array([0]+list(np.random.dirichlet((0.25+count[state][1],0.25+count[state][2],0.2+count[state][3]))))
        state = 2 
        pi[2] = np.array([0]+list(np.random.dirichlet((0.4+count[state][1],0.6+count[state][2])))+[0])
        pi[3] = np.array([0,0,0,1])
        
        pi_filter = update_pi_filter(pi)
        p = pd.concat([p,pd.DataFrame(pi)],axis=0)
        
        # Sanity Check
        if all(np.sum(pi,axis=1))==1 and all(np.sum(pi_filter,axis=1))==1:
            pass
        else:
            raise Exception('probabilities do not sum to 1')      
        print(params['theta']*params['rho'],params['theta']*params['k_sigma2'])
        
        # FFBS
        params['bm'], Sn = gibbs.update_weight_State(hyper_param,pi,pi_filter,params)
        
        # Sanity Check
        if all(Sn[:,0])==0:
            pass
        else:
            raise Exception('The initial state of the particle is not zero')
            
        # Store the Date
        data['theta'].append(params['theta'])
        data['rho'].append(params['rho'])
        data['k_sigma2'].append(params['k_sigma2'])  
        data['E_b'].append(params['theta']*params['rho'])
        data['E_ph'].append(params['theta']*params['k_sigma2'])
        data['B'].append(np.count_nonzero(params['bm']))      
        data['ite'].append(j)
    
        # Timing
        start2 = time.time()
        print(start2-start1,np.count_nonzero(params['bm']),accepted)

        # Save the output
        ## MCMC Chain Parameters
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()]))
        params_data = pd.HDFStore(data2)
        params_data['df{}'.format(j)] = df.loc[j]
        params_data.close()
        ## Emission Distribution
        Ksn = np.where(Sn==1,params['k_sigma2'],0.0)
        estimated_emission = params['theta']*(params['rho']+np.dot(params['bm'],Ksn))*hyper_param['dt']
        estimated_emission_final = pd.DataFrame(dict({'ite':range(N),'est':estimated_emission.reshape(N,)}))
        
        if j%100==0:
            emission_data = pd.HDFStore(data1)
            emission_data['df{}'.format(j)] = pd.DataFrame(estimated_emission)
            emission_data.close()
            ## Trajectory of data
            trajectory_data = pd.HDFStore(data3)
            trajectory_data['df{}'.format(j)] = pd.DataFrame(Sn)
            trajectory_data.close()
            
        # Some postprocessing
#         new_data = dict({'E_b': [params['theta']*params['rho']],'E_ph': [params['theta']*params['k_sigma2']],
#                          'B':[np.count_nonzero(params['bm'])],'ite':[j]})
#         plot_traces(estimated_emission_final,new_data,handle)
    
    # Calculate Acceptance Rate
    Acceptance_rate = accepted/repeats
    print ("Acceptance rate = " + str(Acceptance_rate*100))
    
    return data, Acceptance_rate, p

#### Hyper-parameters:
dt is the exposure time, alfa & beta are hyper-parameters of Theta, alfa1 & beta1 are hyper-parameters of Rho,
alfa2 & beta2 are hyper-parameters of K_sigma2

In [ ]:
repeats = 5000

In [ ]:
hyper_param = {'alfa':1.0,'beta':1.0,'alfa1_proposal':10000,'beta1':1.0,'alfa1_prior':1.0,'alfa2_prior':1.0,
               'alfa2_proposal':10000,'beta2':1.0,'N_particles':15,'gama':1.0,'dt':0.01*.95} 

In [ ]:
M = hyper_param['N_particles']

## Parameters to be calculated by Gibbs Sampler 

In [ ]:
# data1 = 'Wn.h5'
# data2 = 'params.h5'
# params_data = pd.HDFStore(data2)
# params = params_data['df{}'.format(15999)] 
# params = dict(params)

In [ ]:
# params_data.close()

In [ ]:
params = {'theta':1e+1,
          'rho':0.001,
          'k_sigma2':10,
          'bm':np.ones((1,M),dtype=int)}  # bm[1,M]

#### Note: 
The weights for the transition probability distribution are our hyper-parameters and we assume them.

In [ ]:
pi0 = np.array(list(np.random.dirichlet((0.7,0.3)))+[0,0])
pi1 = np.array([0]+list(np.random.dirichlet((0.55,0.25,0.2))))
pi2 = np.array([0]+list(np.random.dirichlet((0.4,0.6)))+[0])
pi3 = np.array([0,0,0,1])

In [ ]:
pi = np.stack((pi0,pi1,pi2,pi3),axis=0)

In [ ]:
pi

In [ ]:
# pi = np.array([[9.95012479e-01, 4.72285706e-03, 2.40596046e-04, 2.40677016e-05],
#        [0.00000000e+00, 8.95880600e-01, 9.46496180e-02, 9.46978182e-03],
#        [0.00000000e+00, 9.46496180e-04, 9.99048684e-01, 4.82001787e-06],
#        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

#### Note: 
We know that the initial states of all particles is known and they are Inactive at the initial time level 

In [ ]:
# data3 = 'trajectory_single_particle.h5'
# trajectory_data = pd.HDFStore(data3)

In [ ]:
Sn = np.zeros([M,N],dtype=np.int)
Sn[:,0] = 0

In [ ]:
# Note: Always double check to make sure wrong transitions are not happening, like more than one transition   
# from state zero to state one and from state one to state zero.
for i in range(M):
    for j in range(1,N):
        Sn[i,j] = np.random.choice(4,p=pi[Sn[i,j-1]])  #Sn[M,N] 
# Sn = pd.DataFrame(Sn)  # This is not efficient from computation point of view

In [ ]:
# Sn = pd.read_csv('trajectory_single_particle.csv').drop('Unnamed: 0',axis=1).values

In [ ]:
# Sn

In [ ]:
Ksn = np.where(Sn==1,params['k_sigma2'],0.0)   # Ksn[M,N]

In [ ]:
def plot_traces(estimated_emission_final,new_data,handle):
    source.stream(new_data)
    source1.stream(estimated_emission_final)
#     doc.add_periodic_callback(__main__)
    push_notebook(handle=handle)

In [ ]:
import time
from line_profiler import LineProfiler
if __name__ == '__main__':
    if os.path.isfile('Wn_single3.h5'):
        os.remove('Wn_single3.h5')
        os.remove('params_single3.h5')
        os.remove('trajectory_single3.h5')
        
    output_notebook()
    source = ColumnDataSource({'E_b':[],'E_ph':[],'B':[],'ite':[]})
    
    s1 = figure(width=250, plot_height=250, title=None)
    s1.scatter(x='ite', y='B', color="navy", alpha=0.5, source=source)

    s2 = figure(width=250, height=250, title=None)
    s2.scatter(x='ite', y='E_b', color="firebrick", alpha=0.5, source=source)

    s3 = figure(width=350, height=250, title=None)
    s3.scatter(x='ite', y='E_ph', color="olive", alpha=0.5, source=source)
    
    source1 = ColumnDataSource(pd.DataFrame(columns=['ite','est']))
    s4 = figure(width=350, height=250, title=None)
    s4.scatter(x='ite', y='est', color="black", alpha=0.5, source=source1)
    s4.scatter(range(N), Wn, color="blue", alpha=0.5)
    
    pl = gridplot([[s1,s2,s3],[s4]], toolbar_location=None)
    handle = show(pl, notebook_handle=True, new='window', notebook_url='localhost:8888')
    
    out = __main__(repeats,M,pi,Sn,Ksn,handle) 
    # Use line-profiler for optimization
#     lp = LineProfiler()
#     lp.add_function(FFBS._FFBS)
#     lp_wrapper = lp(__main__)
#     lp_wrapper(repeats,M,pi,Sn,Ksn,pl)
#     lp.print_stats(), format='pdf'
#------------------------------------------------------
#     %lprun -f __main__  __main__(repeats,M,pi,Sn,Ksn) 

In [ ]:
data1 = 'Wn_single3.h5'
emission_data = pd.HDFStore(data1)
fig = plt.figure(figsize=(4,4)) 
    
plt.style.use('ggplot')
sns.set_style('ticks')
plt.rcParams['font.size'] = 12
for i in range(100,1500,100):
    plt.plot(range(N),emission_data['df{}'.format(i)].iloc[0]) 
plt.scatter(range(N), Wn, color="blue", alpha=0.5)
plt.xlim([0,20000])
# plt.ylim([0,3000])
# plt.ylabel(ylabel)
# plt.xlabel(xlabel)
plt.tight_layout()
sns.despine(offset=10)
# fig.savefig('Emission_distribution.pdf')

In [ ]:
step = 0.01
partitions = [i*step for i in range(round(200./step))]
t = partitions

In [ ]:
np.shape(Sn_true[0,:])

In [ ]:
np.shape(t[:-1])

In [ ]:
Sn_true = pd.read_csv('trajectory_single_particle.csv').drop('Unnamed: 0',axis=1).values

data3 = 'trajectory_single.h5'
traj = pd.HDFStore(data3)
fig = plt.figure(figsize=(4,4)) 
    
plt.style.use('ggplot')
sns.set_style('ticks')
plt.rcParams['font.size'] = 12
for i in range(100,5000,100):
    plt.step(x=t[:-1],y=traj['df{}'.format(i)].iloc[0],color='red') 
plt.step(x=t[:-1],y=Sn_true[0,:], color="blue") 

In [ ]:
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in out[0].items()]))

In [ ]:
h = sns.jointplot(x=df['ite'],y=df['B'], kind='scatter',color='g',s =20,linewidth=2)
h.ax_joint.set_ylim([0,25])

In [ ]:
sns_plot = sns.jointplot(x=df['ite'],y=df['E_b'], kind='scatter',color='g',s =20,linewidth=2)
sns.set(style="darkgrid", color_codes=False)
# sns_plot.savefig('Emission_background.pdf')

In [ ]:
sns_plot = sns.jointplot(x=df['ite'],y=df['E_ph'], kind='scatter',color='g',s =20,linewidth=2)
# sns_plot.ax_joint.set_ylim([0,5000])
sns_plot.savefig('Photon_Emission.pdf')

In [ ]:
d = (scipy.special.kv(hyper_param['alfa']-hyper_param['alfa1_prior'],2*np.sqrt(np.linspace(1,40,1000))*(hyper_param['beta1']*hyper_param['beta2']))/
     np.linspace(1,40,1000)**(1-(hyper_param['alfa']+hyper_param['alfa1_prior'])/2))

In [ ]:
n, bins, patches = plt.hist(out[0]['E_b'], 50, facecolor='blue')
plt.plot(d*10000,color='k')
plt.xlim([0,40])
# plt.ylim([0,1])
# plt.axvline(1000, color='r')

In [ ]:
n, bins, patches = plt.hist(out[0]['E_ph'], 50, facecolor='blue')
plt.axvline(1e+3, color='r')

In [ ]:
plt.step(x=t[:-1],y=Sn[0,:])

In [ ]:
out1 = out[2] 
pi_true = np.array([[9.95012479e-01, 4.72285706e-03, 2.40596046e-04, 2.40677016e-05],
                    [0.00000000e+00, 8.95880600e-01, 9.46496180e-02, 9.46978182e-03],
                    [0.00000000e+00, 9.46496180e-04, 9.99048684e-01, 4.82001787e-06],
                    [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
d = dict([(index,0) for index,v in np.ndenumerate(pi)])
for index, x in np.ndenumerate(pi):
    i = index[0]
    j = index[1]
    d[index] = list(out1.iloc[i::4,j])
    
def plot_hist(dic, pi_true):
    fig = plt.figure(figsize=(16,16)) 
    plt.style.use('ggplot')
    sns.set_style('ticks')
    plt.rcParams['font.size'] = 12

    nn = 1
    for i, x in np.ndenumerate(pi_true):
        ax = fig.add_subplot(4,4,nn)
        n, bins, patches = plt.hist(dic[i], 10, facecolor='blue')
        plt.axvline(pi_true[i], color='r')
#         ax.set_title(r'$particle-ID: {}$'.format(str(i)))
#         ax.legend().set_visible(False)
        sns.despine(offset=12,ax=ax,trim=True)
        nn = nn+1

    # plt.subplots_adjust(top=0.92,bottom=0.08,left=0.1,right=0.95,wspace=0.6,hspace=0.6)
    plt.tight_layout()
    plt.show() 
    fig.savefig('Probability_distribution.pdf')
    return None

In [ ]:
plot_hist(d, pi_true)